In [35]:
import os
import sys
import re
import glob
import subprocess
import datetime
import time
import pytz
import exifread
import zipfile
# from osgeo import gdal

import ee
import ee.cli
import ee.cli.commands
import ee.cli.utils

In [49]:
def run(cmd):
    print(cmd)
    subprocess.call(cmd)
    
def extractAll(zipName):
    z = zipfile.PyZipFile(zipName)
    for f in z.namelist():
        if f.endswith('/'):
            os.makedirs(f)
        else:
            z.extract(f)

In [37]:
ee.Initialize()
ee_config = ee.cli.utils.CommandLineConfig()

In [52]:
input_dir = r'P:/11202200.005-kpp-cip2018/data/Satellitedataportaal'
os.chdir(input_dir)

rapideye = True
triplesat = False

local = pytz.timezone("Europe/Amsterdam")


In [66]:
# unzip all .zip files

# find all .tif files
if rapideye:
    files = glob.glob('RapidEye/**/**/*.tif', recursive=True)
    zippedfiles = glob.glob('RapidEye/**/**/*.zip', recursive=True)
elif triplesat:
    files = glob.glob('TripleSat/**/**/*.tif', recursive=True)
    zippedfiles = glob.glob('TripleSat/**/**/*.zip', recursive=True)
else:
    files = None
    zippedfiles = None
    print('Can only upload one satellite at a time. Please set rapideye or triplesat only.')

# files = glob.glob('TripleSat/**/*.tif')

print(['Number of files to upload as assets = '+str(len(files))])


['Number of files to upload as assets = 200']


In [72]:
filename = os.path.basename(zippedfiles[1])
filename_no_ext = os.path.splitext(zippedfiles[1])[0]
dir_path = os.path.dirname(os.path.abspath(zippedfiles[1]))
filename_unzipped = os.path.join(dir_path,filename_no_ext+'.tif')
test_unzipped = os.path.exists(filename_unzipped)
print(test_unzipped)
if test_unzipped: 
    print(['tif file exists for ' + filename])
else:
    print(['tif file DOES NOT exist for ' + filename])

False
['tif file DOES NOT exist for 20170331_3163722_RE_5m_8bit_RGB_RD.zip']


In [50]:

# z = zipfile.ZipFile(zippedfiles)

for j in zippedfiles:
    filename = os.path.basename(j)
    filename_no_ext = os.path.splitext(j)[0]
    dir_path = os.path.dirname(os.path.abspath(files[1]))
    unzipped = 
    if (dir_path,filename,'.tif'):
    else:
        extractAll(j)


# if __name__ == '__main__':
#     zipList = ['one.zip', 'two.zip', 'three.zip']
#     for zip in zipList:
#         extractAll(zipName)

In [28]:
for i, f in enumerate(files[1:3]):
    print('Processing file ' + f + ', file index: ' + str(i))

    # extract time in UTC
    fh = open(f, 'rb')
    tags = exifread.process_file(fh, stop_tag="Image DateTime")
    datestring = str(tags["Image DateTime"].values)
    t = datetime.datetime.strptime(datestring, '%Y:%m:%d %H:%M:%S')
    local_t = local.localize(t, is_dst=None)
    utc_t = local_t.astimezone(pytz.utc)
    time_start = utc_t.strftime('%Y-%m-%dT%H:%M:%S')

    # parse file names
    filename = os.path.basename(f)
    filename_no_ext = os.path.splitext(filename)[0]

    # get nodata value ... UGLY, UGLY code!
    nodata_value = -99999999
    with open(f, encoding="latin_1") as asc:
         for line in asc:
            if "nodata_value" in line.lower():
                nodata_value = line.split()[1]
                break
    if rapideye:
    # copy to GCS 
        run(r"C:\Users\Wilson\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\gsutil.cmd cp {0} gs://eo-bathymetry/rapideye/{1}".format(os.path.join(input_dir,f),filename))
    elif triplesat:
        run(r"C:\Users\Wilson\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\gsutil.cmd cp {0} gs://eo-bathymetry/triplesat/{1}".format(os.path.join(input_dir,f),filename))
    else:
        print('Can only upload one satellite at a time. Please set rapideye or triplesat only.')
        break
        
    # upload to GEE
    retry_count = 0

#     while True:
#         if rapideye:
#             run("earthengine upload image --wait --asset_id=users/rogersckw9/rapideye/{1} --nodata_value={0} gs://eo-bathymetry/rapideye/{1}".format(nodata_value, filename))
#         else:
#             run("earthengine upload image --wait --asset_id=users/rogersckw9/triplesat/{1} --nodata_value={0} gs://eo-bathymetry/triplesat/{1}".format(nodata_value, filename))
            
        # check last task status
        tasks = ee.data.getTaskList()
        task_state = None
        for task in tasks:
            task_status = ee.data.getTaskStatus([task['id']])
            task_state = task_status[0]['state']
            print(task_status)
            break

        if task_state != 'FAILED':
            break # done
        else:
            retry_count += 1
            print('Retrying upload ' + str(retry_count) + ' ...')

        if retry_count > 10:
            print('Maximum number of retry reached, exiting ...')
            sys.exit(0)

    # set time
    if rapideye:
        run("earthengine asset set --time_start {0} users/rogersckw9/rapideye/{1}".format(time_start, filename_no_ext))
    else:
        run("earthengine asset set --time_start {0} users/rogersckw9/triplesat/{1}".format(time_start, filename_no_ext))

Processing file RapidEye\2017_03\20170325\20170325_3163722_RE_5m_8bit_RGB_RD.tif, file index: 0
C:\Users\Wilson\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\gsutil.cmd cp P:/11202200.005-kpp-cip2018/data/Satellitedataportaal\RapidEye\2017_03\20170325\20170325_3163722_RE_5m_8bit_RGB_RD.tif gs://eo-bathymetry/rapideye/20170325_3163722_RE_5m_8bit_RGB_RD.tif
earthengine upload image --wait --asset_id=users/rogersckw9/rapideye/20170325_3163722_RE_5m_8bit_RGB_RD.tif --nodata_value=-99999999 gs://eo-bathymetry/rapideye/20170325_3163722_RE_5m_8bit_RGB_RD.tif
[{'id': 'IHJ2XPQJZSPO2GZHRAFA27PQ', 'state': 'COMPLETED', 'creation_timestamp_ms': 1522415523002, 'update_timestamp_ms': 1522415627887, 'description': 'Asset ingestion: users/rogersckw9/20170327_3163722_RE_5m_8bit_RGB_RD', 'task_type': 'INGEST', 'output_url': ['https://code.earthengine.google.com/?asset=users/rogersckw9/20170327_3163722_RE_5m_8bit_RGB_RD'], 'start_timestamp_ms': 1522415526221}]
earthengine asset set --time_start 201